In [ ]:
latex_parse_overview = dict()
for sections in all_articles[0]['latex_parse']['body_text']:
    if sections['section'] in latex_parse_overview:
        # если есть дублирование, такое бывает у первых частo
        if latex_parse_overview[sections['section']] == sections:
            continue
        else:
            latex_parse_overview[sections['section']]['text'].append(sections['text'])
            latex_parse_overview[sections['section']]['cite_spans'].append(sections['cite_spans'])
            latex_parse_overview[sections['section']]['cite_span_lens'].append(len(sections['cite_spans']))
            latex_parse_overview[sections['section']]['cite_spans_start'].append(sections['section'])
            latex_parse_overview[sections['section']]['section'].append(sections['section'])
#             latex_parse_overview[sections['section']]['section'].append(sections['section'])
    else:
        latex_parse_overview[sections['section']] = {'text':[sections['text']],   'cite_spans':[sections['cite_spans']],
                                                      'cite_span_lens':[len(sections['cite_spans'])],
                                                      'cite_spans_start':list(map(lambda x: int(x['start']),sections['cite_spans'])),
                                                      'section':[sections['section']], 
                                                      'bib_entries':all_articles[0]['latex_parse']}
       
    
for sections in latex_parse_overview:
    sections_dict = latex_parse_overview[sections]
    print(sections,len(sections_dict['cite_spans']),sections_dict['cite_span_lens'])
    print(sections_dict)
    print(10*'==')

In [1]:
import json
from tqdm import tqdm_notebook
import gzip
import pandas as pd
import sys

In [2]:
import json
from typing import Dict, List


def get_citation_contexts(paper: Dict, toks_in_context=10) -> List[Dict]:
    """
    Retrieve citation contexts from GORC paper
    :param paper:
    :param toks_in_context:
    :return:
    """
    if not paper:
        return []

    if not paper['grobid_parse']:
        return []

    if not paper['grobid_parse']['body_text']:
        return []

    contexts = []

    for paragraph in paper['grobid_parse']['body_text']:
        for cite_span in paragraph['cite_spans']:
            # get cited paper id, skip if none
            cite_ref = cite_span['ref_id']
            cited_paper_id = None
            if cite_ref in paper['grobid_parse']['bib_entries']:
                cited_paper_id = paper['grobid_parse']['bib_entries'][cite_ref]['links']
            if not cited_paper_id:
                continue

            # get pre and post tokens
            pre_span_tokens = paragraph['text'][:cite_span['start']].split(' ')[-toks_in_context:]
            post_span_tokens = paragraph['text'][cite_span['end']:].split(' ')[:toks_in_context]
            pre_string = ' '.join(pre_span_tokens)
            post_string = ' '.join(post_span_tokens)
            full_context = pre_string + cite_span['text'] + post_string

            contexts.append({
                "paper_id": paper['paper_id'],
                "context_string": full_context,
                "cite_start": len(pre_string),
                "cite_end": len(pre_string) + len(cite_span['text']),
                "cite_str": cite_span['text'],
                "cited_paper_id": cited_paper_id
            })

    return contexts


In [3]:
EXAMPLE_DATA_FILE = 'data/example_papers.jsonl'

In [4]:
all_contexts = []
all_papers = []
context_dict = dict()
with open(EXAMPLE_DATA_FILE, 'r') as f:
    for line in f:
        gorc_obj = json.loads(line)
        all_papers.append(gorc_obj)
        all_contexts += get_citation_contexts(gorc_obj)

In [5]:
from os import walk

f_zips = []
for (dirpath, dirnames, filenames) in walk('../../gorc/'):
    f_zips.extend(filenames)
    break

In [6]:
len(f_zips),f_zips

(10002,
 ['0.jsonl.gz',
  '1.jsonl.gz',
  '10.jsonl.gz',
  '100.jsonl.gz',
  '1000.jsonl.gz',
  '1001.jsonl.gz',
  '1002.jsonl.gz',
  '1003.jsonl.gz',
  '1004.jsonl.gz',
  '1005.jsonl.gz',
  '1006.jsonl.gz',
  '1007.jsonl.gz',
  '1008.jsonl.gz',
  '1009.jsonl.gz',
  '101.jsonl.gz',
  '1010.jsonl.gz',
  '1011.jsonl.gz',
  '1012.jsonl.gz',
  '1013.jsonl.gz',
  '1014.jsonl.gz',
  '1015.jsonl.gz',
  '1016.jsonl.gz',
  '1017.jsonl.gz',
  '1018.jsonl.gz',
  '1019.jsonl.gz',
  '102.jsonl.gz',
  '1020.jsonl.gz',
  '1021.jsonl.gz',
  '1022.jsonl.gz',
  '1023.jsonl.gz',
  '1024.jsonl.gz',
  '1025.jsonl.gz',
  '1026.jsonl.gz',
  '1027.jsonl.gz',
  '1028.jsonl.gz',
  '1029.jsonl.gz',
  '103.jsonl.gz',
  '1030.jsonl.gz',
  '1031.jsonl.gz',
  '1032.jsonl.gz',
  '1033.jsonl.gz',
  '1034.jsonl.gz',
  '1035.jsonl.gz',
  '1036.jsonl.gz',
  '1037.jsonl.gz',
  '1038.jsonl.gz',
  '1039.jsonl.gz',
  '104.jsonl.gz',
  '1040.jsonl.gz',
  '1041.jsonl.gz',
  '1042.jsonl.gz',
  '1043.jsonl.gz',
  '1044.jsonl.gz'

In [7]:
[file for file in f_zips if '.gz' not in file]

['s2orc-master.zip']

In [12]:
all_papers[0]['grobid_parse']['bib_entries']

{'BIBREF0': {'ref_id': 'b0',
  'title': 'Tilt change recorded by broadband seismometer prior to 476 small phreatic explosion of Meakan-dake volcano',
  'authors': [{'first': 'H', 'middle': [], 'last': 'Aoyama', 'suffix': ''},
   {'first': 'H', 'middle': [], 'last': 'Oshima', 'suffix': ''}],
  'year': 2008,
  'venue': 'Geophys. Res. Lett',
  'volume': '477',
  'issn': '6',
  'pages': '',
  'other_ids': {},
  'links': None},
 'BIBREF1': {'ref_id': 'b1',
  'title': 'Very long period conduit oscillations induced by rockfalls at 479',
  'authors': [{'first': 'B', 'middle': [], 'last': 'Chouet', 'suffix': ''},
   {'first': 'P', 'middle': [], 'last': 'Dawson', 'suffix': ''}],
  'year': 2013,
  'venue': '',
  'volume': '',
  'issn': '',
  'pages': '',
  'other_ids': {},
  'links': None},
 'BIBREF4': {'ref_id': 'b4',
  'title': 'Source mechanisms of explosions at Stromboli Volcano',
  'authors': [{'first': 'G', 'middle': [], 'last': 'Milana', 'suffix': ''},
   {'first': 'R', 'middle': [], 'last

In [13]:
all_papers[0]['grobid_parse']['body_text'][0].keys()

dict_keys(['text', 'cite_spans', 'ref_spans', 'eq_spans', 'section'])

In [14]:
all_articles = all_papers

In [15]:
sum([1 for paper in all_papers if paper['metadata']['acl_id']!=None])


1

In [16]:
with open("acl_only_json_list_10000.json", "r") as read_file:
    all_articles = json.load(read_file)
print(len(all_articles))
read_file.close()

41660


## Анализ подборки

### проверка наличия названия секции

In [17]:
all_articles[0].keys()

dict_keys(['paper_id', 'metadata', 's2_pdf_hash', 'grobid_parse', 'latex_parse'])

In [18]:
acl_paper_ids = [article['paper_id'] for article in all_articles]

##### количество  всех статьей

In [21]:
len([article['paper_id'] for article in all_articles if article['grobid_parse']['body_text'] or (article['latex_parse'] and article['latex_parse']['body_text'])])

41439

#### проверка наличия текста и названия секций во всех статьях в grobid части

In [22]:
len([article['paper_id'] for article in all_articles if article['grobid_parse']['body_text']])

41439

In [23]:
acl_ids_not_bofy_text = [article['paper_id'] for article in all_articles if not article['grobid_parse']['body_text']]

In [24]:
for num,paper_id in enumerate(acl_ids_not_bofy_text):
    if num == 2:
        break
    id_lst = acl_paper_ids.index(paper_id)
    print(id_lst,all_articles[id_lst])
    print(10*'==')

156 {'paper_id': '60131735', 'metadata': {'title': 'Breadth and D e p t h of Semant ic Lexicons Proceedings of a Workshop Sponsored by the Special Interest Group on the Lexicon of the Associat ion for Computat ional Linguistics', 'authors': [{'first': 'Evelyne', 'middle': [], 'last': 'Viegas', 'suffix': ''}], 'abstract': 'Preface. Contributors. Introduction: E. Viegas. I. Lexical Rules and Underspecification. II. Breadth of Semantic Lexicons. III. Depth of Semantic Lexicons. IV. Lexical Semantics and Pragmatics. Subject Index. Author Index.', 'year': '1999', 'arxiv_id': None, 'acl_id': 'W96-0300', 'pmc_id': None, 'pubmed_id': None, 'doi': '10.1007/978-94-017-0952-1', 'venue': 'Text, Speech and Language Technology', 'journal': 'Text, Speech and Language Technology'}, 's2_pdf_hash': 'fef5115fca1124e5f994cd49414c743ca005b853', 'grobid_parse': {'abstract': [], 'body_text': [], 'ref_entries': {'TABREF0': {'text': 'Introduction', 'latex': None, 'type': 'table'}}, 'bib_entries': {}}, 'latex_p

In [25]:
all_articles[186]

{'paper_id': '7473534',
 'metadata': {'title': 'Interfacing Ontologies and Lexical Resources',
  'authors': [{'first': 'Laurent',
    'middle': [],
    'last': 'Prévot',
    'suffix': ''},
   {'first': 'Stefano', 'middle': [], 'last': 'Borgo', 'suffix': ''},
   {'first': 'Alessandro', 'middle': [], 'last': 'Oltramari', 'suffix': ''}],
  'abstract': 'During the last few years, a number of works aiming at interfacing ontologies and lexical resources have been initiated. This paper aims at clarifying the current picture of this domain. It compares ontologies built following different methodologies and analyses their combination with lexical resources. A point defended in the paper is that different methodologies lead to very different characteristics for the resulting resources. We classify these methodologies show how actual projects fit into this classification.',
  'year': '2005',
  'arxiv_id': None,
  'acl_id': 'I05-7013',
  'pmc_id': None,
  'pubmed_id': None,
  'doi': '10.1017/CBO97

In [28]:
article_with_sect = dict()
for article in all_articles:
    for sections in article['grobid_parse']['body_text']:
        if sections['section']:
            if article['paper_id'] in article_with_sect:
                article_with_sect[article['paper_id']] +=1
            else:
                article_with_sect[article['paper_id']] = 1

In [29]:
article_with_sect

{}

###### как видим нет названия секций у grobid_parse

In [30]:
all_articles[1]

{'paper_id': '14472576',
 'metadata': {'title': 'Building a Semantic Parser Overnight',
  'authors': [{'first': 'Yushi', 'middle': [], 'last': 'Wang', 'suffix': ''},
   {'first': 'Jonathan', 'middle': [], 'last': 'Berant', 'suffix': ''},
   {'first': 'Percy', 'middle': [], 'last': 'Liang', 'suffix': ''}],
  'abstract': 'How do we build a semantic parser in a new domain starting with zero training examples? We introduce a new methodology for this setting: First, we use a simple grammar to generate logical forms paired with canonical utterances. The logical forms are meant to cover the desired set of compositional operators, and the canonical utterances are meant to capture the meaning of the logical forms (although clumsily). We then use crowdsourcing to paraphrase these canonical utterances into natural utterances. The resulting data is used to train the semantic parser. We further study the role of compositionality in the resulting paraphrases. Finally, we test our methodology on seve

In [33]:
# article_with_sect = dict()
for num,article in enumerate(all_articles):
    if num == 2:
        break
    for cnt_sect,sections in enumerate(article['grobid_parse']['body_text']):
        if sections['cite_spans']:
            print(f'номер секции = {cnt_sect} | кол-во ссылок',len(sections['cite_spans']))
            for cnt_cite,cite in enumerate(sections['cite_spans']):
                print(cnt_cite,cite)
            print(sections['text'])
            print('----')
    print(10*'==')
#             if article['paper_id'] in article_with_sect:
#                 article_with_sect[article['paper_id']] +=1
#             else:
#                 article_with_sect[article['paper_id']] = 1

номер секции = 0 | кол-во ссылок 13
0 {'start': 192, 'end': 216, 'text': '(Goldstein et al., 2000;', 'latex': None, 'ref_id': 'BIBREF6'}
1 {'start': 217, 'end': 239, 'text': 'Erkan and Radev, 2004;', 'latex': None, 'ref_id': 'BIBREF4'}
2 {'start': 240, 'end': 257, 'text': 'Wan et al., 2007;', 'latex': None, 'ref_id': 'BIBREF19'}
3 {'start': 258, 'end': 284, 'text': 'Nenkova and McKeown, 2012;', 'latex': None, 'ref_id': 'BIBREF16'}
4 {'start': 285, 'end': 302, 'text': 'Min et al., 2012;', 'latex': None, 'ref_id': 'BIBREF15'}
5 {'start': 303, 'end': 319, 'text': 'Li et al., 2017)', 'latex': None, 'ref_id': 'BIBREF11'}
6 {'start': 773, 'end': 797, 'text': '(Project Code: 14203414)', 'latex': None, 'ref_id': None}
7 {'start': 2288, 'end': 2305, 'text': '(Hu et al., 2008;', 'latex': None, 'ref_id': 'BIBREF7'}
8 {'start': 2306, 'end': 2324, 'text': 'Yang et al., 2011)', 'latex': None, 'ref_id': 'BIBREF22'}
9 {'start': 2582, 'end': 2598, 'text': 'Li et al. (2015)', 'latex': None, 'ref_id': 'B

To evaluate the performance of our dataset and the proposed framework RAVAESum for RA-MDS, we compare our model with the following methods:• RA-Sparse : It is a framework to tackle the RA-MDS problem. A sparse-coding-based method is used to calculate the salience of the news sentences by jointly considering news documents and reader comments.• Lead (Wasson, 1998) : It ranks the news sentences chronologically and extracts the leading sentences one by one until the length limit.• Centroid (Radev et al., 2000) : It summarizes clusters of news articles automatically grouped by a topic detection system, and then it uses information from the centroids of the clusters to select sentences.• LexRank (Erkan and Radev, 2004) and TextRank (Mihalcea and Tarau, 2004) : Both methods are graph-based unsupervised framework for sentence salience estimation based on PageRank algorithm.• Concept : It generates abstractive summaries using phrase-based optimization framework with concept weight as salience 

Our logical forms are represented in lambda DCS, a logical language where composition operates on sets rather than truth values. Here we give a brief description; see Liang (2013) for details.Every logical form z in this paper is either a unary (denoting a set of entities) or a binary (denoting a set of entity-pairs). In the base case, each entity e (e.g., 2015) is a unary denoting the singleton set: e w = {e}; and each property p (e.g., publicationDate) is a binary denoting all entitypairs (e 1 , e 2 ) that satisfy the property p. Unaries and binaries can be composed: Given a binary b and unary u, the join b.u denotes all entities e 1 for which there exists an e 2 ∈ u w with (e 1 , e 2 ) ∈ b w . For example, publicationDate.2015 denote entities published in 2015.The intersection u 1 u 2 , union u 1 u 2 , complement ¬u denote the corresponding set operations on the denotations. We let R(b) denote the reversal of b: (e 1 , e 2 ) ∈ b w iff (e 2 , e 1 ) ∈ R(b) w . This allows us to define

all_results[0] - Reader-Aware Multi-Document Summarization: An Enhanced Model and The First Dataset∗



Result 

- 0 {'start': 192, 'end': 216, 'text': '(Goldstein et al., 2000;', 'latex': None, 'ref_id': 'BIBREF6'}
- 1 {'start': 217, 'end': 239, 'text': 'Erkan and Radev, 2004;', 'latex': None, 'ref_id': 'BIBREF4'}
- 2 {'start': 240, 'end': 257, 'text': 'Wan et al., 2007;', 'latex': None, 'ref_id': 'BIBREF19'}
- 3 {'start': 258, 'end': 284, 'text': 'Nenkova and McKeown, 2012;', 'latex': None, 'ref_id': 'BIBREF16'}
- 4 {'start': 285, 'end': 302, 'text': 'Min et al., 2012;', 'latex': None, 'ref_id': 'BIBREF15'}
- 5 {'start': 303, 'end': 319, 'text': 'Li et al., 2017)', 'latex': None, 'ref_id': 'BIBREF11'}

True 

- 0 (Goldstein et al., 2000; 
- 1 Erkan and Radev,2004; 
- 2 Wan et al., 2007; 
- 3 Nenkova and McKeown, 2012; 
- 4 Min et al., 2012; 
- 5 Bing et al., 2015; 
- 6 Li et al.,2017)


Result

- {'start': 971, 'end': 987, 'text': 'Li et al. (2015)', 'latex': None, 'ref_id': 'BIBREF2'}

True

- Woodsend and Lapata (2012), Bing et al. (2015), and Li et al. (2015).

**При большом перечислении подряд ссылок, GROBID не выделяет предпоследнюю ссылку**

**Также он не срабатывает на части ссылок**

#### проверка наличия текста и названия секций во всех статьях в latex части

In [34]:
len([article['paper_id'] for article in all_articles if article['latex_parse'] and article['latex_parse']['body_text']])

4039

In [35]:
acl_ids_not_body_text_tex = [article['paper_id'] for article in all_articles if not ( article['latex_parse'] and article['latex_parse']['body_text'])]

In [36]:
len(acl_ids_not_body_text_tex)

37621

In [37]:
for num,paper_id in enumerate(acl_ids_not_body_text_tex):
    if num == 2:
        break
    id_lst = acl_paper_ids.index(paper_id)
    print(id_lst,all_articles[id_lst]['latex_parse'])
    print(10*'==')

1 None
2 None


In [40]:
{k: all_articles[0]['latex_parse']['bib_entries'][k] for k in sorted(all_articles[0]['latex_parse']['bib_entries'])}

{'BIBREF0': {'ref_id': 'BIBREF0',
  'title': 'Multi-document summarization by sentence extraction',
  'authors': [{'first': 'Jade',
    'middle': [],
    'last': 'Goldstein',
    'suffix': ''},
   {'first': 'Vibhu', 'middle': [], 'last': 'Mittal', 'suffix': ''},
   {'first': 'Jaime', 'middle': [], 'last': 'Carbonell', 'suffix': ''},
   {'first': 'Mark', 'middle': [], 'last': 'Kantrowitz', 'suffix': ''}],
  'year': 2000,
  'venue': 'NAACL-ANLPWorkshop',
  'volume': '',
  'issn': '',
  'pages': '40--48',
  'other_ids': {},
  'links': '8294822'},
 'BIBREF1': {'ref_id': 'BIBREF1',
  'title': 'Lexpagerank: Prestige in multi-document text summarization',
  'authors': [{'first': 'Günes', 'middle': [], 'last': 'Erkan', 'suffix': ''},
   {'first': '', 'middle': [], 'last': 'Dragomir R Radev', 'suffix': ''}],
  'year': 2004,
  'venue': 'EMNLP',
  'volume': '4',
  'issn': '',
  'pages': '365--371',
  'other_ids': {},
  'links': '10418456'},
 'BIBREF10': {'ref_id': 'BIBREF10',
  'title': 'Auto-enc

In [42]:
article_with_sect_latex = dict()
for article in all_articles:
    if article['latex_parse'] and article['latex_parse']['body_text']: 
        for sections in article['latex_parse']['body_text']:
            if sections['section']:
                if article['paper_id'] in article_with_sect_latex:
                    article_with_sect_latex[article['paper_id']] +=1
                else:
                    article_with_sect_latex[article['paper_id']] = 1

##### Количество статей, в которых есть названия секций

In [43]:
len(article_with_sect_latex)

3657

Проверка ситуации, когда есть **latex_parse**, но  нет **grobid_parse**

In [47]:
for num_art ,article in enumerate(all_articles):
    if article['latex_parse'] and article['latex_parse']['body_text']:
        if len(article['grobid_parse']['body_text'])==0:
            print(num_art,articcle['paper_id'])

Не бывает

### Выделение обзорной части статьи

1. Самый простой принцип построения - по максимальному количеству ссылок в абзаце:
 - **Решение**:
    - подсчитать количество ссылок в каждой секции
    - выбрать секцию с максимальным количеством ссылок (возможно ещё оставить ещё 1 секцию, в которой количество ссылок было больше половины чем в максимальной)
    - для latex статей надо объединить текст одинаковых секций в 1 абзац
 - **Критерий**:
    -  в части latex публикаций есть названия секций => после выделения обзорных часте можно посмотреть какие секции выделились: какие топ-3, сделать просмотр глазами и после этого решать что делать дальше.
    - возможно логично сохранять для 2 максимального текста название статей
    

In [48]:
grobid_parse_overview = dict()
for num_sec,sections in enumerate(all_articles[0]['grobid_parse']['body_text']):
    grobid_parse_overview[num_sec] = sections
    print(len(grobid_parse_overview[num_sec]['cite_spans']),sections)
    print(10*'==')

13 {'text': 'The goal of multi-document summarization (MDS) is to automatically generate a brief, wellorganized summary for a topic which describes an event with a set of documents from different sources. (Goldstein et al., 2000; Erkan and Radev, 2004; Wan et al., 2007; Nenkova and McKeown, 2012; Min et al., 2012; Li et al., 2017) . In the typical setting of MDS, the input is a set of news documents about the same topic. The output summary is a piece of short text document containing several sentences, generated only based on the input original documents.With the development of social media and mobile equipments, more and more user generated * The work described in this paper is supported by a grant from the Grant Council of the Hong Kong Special Administrative Region, China (Project Code: 14203414) .1 http://www.se.cuhk.edu.hk/˜textmine/ dataset/ra-mds/ NEWS: The most important announcements from Google\'s big developers\' conference content is available. Figure 1 is a snapshot of rea

0 {'text': 'In this section, we describe the preparation process of the dataset. Then we provide some properties and statistics.', 'cite_spans': [], 'ref_spans': [], 'eq_spans': [], 'section': None}
0 {'text': 'The definition of the terminology related to the dataset is given as follows. 6 Topic: A topic refers to an event and it is composed of a set of news documents from different sources. Document: A news article describing some aspects of the topic. The set of documents in the same topic typically span a period, say a few days. Category: Each topic belongs to a category. There are 6 predefined categories: (1) Accidents and Natural Disasters, (2) Attacks (Criminal/Terrorist), (3) New Technology, (4) Health and Safety, (5) Endangered Resources, and (6) Investigations and Trials (Criminal/Legal/Other). Aspect: Each category has a set of predefined aspects. Each aspect describes one important element of an event. For example, for the category "Accidents and Natural Disasters", the aspe

In [49]:
grobid_parse_overview

{0: {'text': 'The goal of multi-document summarization (MDS) is to automatically generate a brief, wellorganized summary for a topic which describes an event with a set of documents from different sources. (Goldstein et al., 2000; Erkan and Radev, 2004; Wan et al., 2007; Nenkova and McKeown, 2012; Min et al., 2012; Li et al., 2017) . In the typical setting of MDS, the input is a set of news documents about the same topic. The output summary is a piece of short text document containing several sentences, generated only based on the input original documents.With the development of social media and mobile equipments, more and more user generated * The work described in this paper is supported by a grant from the Grant Council of the Hong Kong Special Administrative Region, China (Project Code: 14203414) .1 http://www.se.cuhk.edu.hk/˜textmine/ dataset/ra-mds/ NEWS: The most important announcements from Google\'s big developers\' conference content is available. Figure 1 is a snapshot of re

In [50]:
latex_parse_overview = dict()
for sections in all_articles[0]['latex_parse']['body_text']:
    latex_parse_overview[sections['section']] = sections
    print(sections['section'],len(sections['cite_spans']))
    print(sections)
    print(10*'==')

Introduction 7
{'text': 'The goal of multi-document summarization (MDS) is to automatically generate a brief, well-organized summary for a topic which describes an event with a set of documents from different sources. BIBREF0 , BIBREF1 , BIBREF2 , BIBREF3 , BIBREF4 , BIBREF5 , BIBREF6 . In the typical setting of MDS, the input is a set of news documents about the same topic. The output summary is a piece of short text document containing several sentences, generated only based on the input original documents.', 'cite_spans': [{'start': 193, 'end': 200, 'text': None, 'latex': None, 'ref_id': 'BIBREF0'}, {'start': 203, 'end': 210, 'text': None, 'latex': None, 'ref_id': 'BIBREF1'}, {'start': 213, 'end': 220, 'text': None, 'latex': None, 'ref_id': 'BIBREF2'}, {'start': 223, 'end': 230, 'text': None, 'latex': None, 'ref_id': 'BIBREF3'}, {'start': 233, 'end': 240, 'text': None, 'latex': None, 'ref_id': 'BIBREF4'}, {'start': 243, 'end': 250, 'text': None, 'latex': None, 'ref_id': 'BIBREF5'}, 

{'text': 'VAESum BIBREF6 employs an alignment mechanism BIBREF12 , BIBREF13 to recall the lost detailed information from the input sentence. Inspired this idea, we design a jointly weighted alignment mechanism by considering the news sentence and the comment sentence simultaneously. For each decoder hidden state INLINEFORM0 , we align it with each news encoder hidden state INLINEFORM1 by an alignment vector INLINEFORM2 . We also align it with each comments encoder hidden state INLINEFORM3 by an alignment vector INLINEFORM4 . In order to filter the noisy information from the comments, we again employ the comment weight INLINEFORM5 to adjust the alignment vector of comments: DISPLAYFORM0 ', 'cite_spans': [{'start': 7, 'end': 14, 'text': None, 'latex': None, 'ref_id': 'BIBREF6'}, {'start': 46, 'end': 54, 'text': None, 'latex': None, 'ref_id': 'BIBREF12'}, {'start': 57, 'end': 65, 'text': None, 'latex': None, 'ref_id': 'BIBREF13'}], 'ref_spans': [], 'eq_spans': [{'start': 304, 'end': 315, 

{'text': 'Each topic is assigned to 4 experts, who are major in journalism, to conduct the summary writing. The task of summary writing is divided into two phases, namely, aspect facet identification, and summary generation. For the aspect facet identification, the experts read and digested all the news documents and reader comments under the topic. Then for each aspect, the experts extracted the related facets from the news document. The summaries were generated based on the annotated aspect facets. When selecting facets, one consideration is those facets that are popular in both news documents and reader comments have higher priority. Next, the facets that are popular in news documents have the next priority. The generated summary should cover as many aspects as possible, and should be well-organized using complete sentences with a length restriction of 100 words.', 'cite_spans': [], 'ref_spans': [], 'eq_spans': [], 'section': 'Data Collection'}
Data Collection 0
{'text': 'After fini

In [51]:
for article in all_articles[0]['latex_parse']['body_text']:
    print(article['section'],len(article['cite_spans']))
    print(article['text'])
    print(10*'==')

Introduction 7
The goal of multi-document summarization (MDS) is to automatically generate a brief, well-organized summary for a topic which describes an event with a set of documents from different sources. BIBREF0 , BIBREF1 , BIBREF2 , BIBREF3 , BIBREF4 , BIBREF5 , BIBREF6 . In the typical setting of MDS, the input is a set of news documents about the same topic. The output summary is a piece of short text document containing several sentences, generated only based on the input original documents.
Introduction 0
With the development of social media and mobile equipments, more and more user generated content is available. Figure FIGREF2 is a snapshot of reader comments under the news report “The most important announcements from Google's big developers' conference”. The content of the original news report talks about some new products based on AI techniques. The news report generally conveys an enthusiastic tone. However, while some readers share similar enthusiasms, some others expre

Each topic is assigned to 4 experts, who are major in journalism, to conduct the summary writing. The task of summary writing is divided into two phases, namely, aspect facet identification, and summary generation. For the aspect facet identification, the experts read and digested all the news documents and reader comments under the topic. Then for each aspect, the experts extracted the related facets from the news document. The summaries were generated based on the annotated aspect facets. When selecting facets, one consideration is those facets that are popular in both news documents and reader comments have higher priority. Next, the facets that are popular in news documents have the next priority. The generated summary should cover as many aspects as possible, and should be well-organized using complete sentences with a length restriction of 100 words.
Data Collection 0
After finishing the summary writing procedure, we employed another expert for scrutinizing the summaries. Each su

In [52]:
all_articles[0]['latex_parse']['body_text'][0].keys()

dict_keys(['text', 'cite_spans', 'ref_spans', 'eq_spans', 'section'])

In [53]:
latex_parse_overview = dict()
for sections in all_articles[0]['latex_parse']['body_text']:
    if sections['section'] in latex_parse_overview:
        # если есть дублирование, такое бывает у первых часте
        if latex_parse_overview[sections['section']] == sections:
            continue
        else:
            latex_parse_overview[sections['section']]['text'].append(sections['text'])
            latex_parse_overview[sections['section']]['cite_spans'].append(sections['cite_spans'])
            latex_parse_overview[sections['section']]['cite_span_lens'].append(len(sections['cite_spans']))
            latex_parse_overview[sections['section']]['section'].append(sections['section'])
#             latex_parse_overview[sections['section']]['section'].append(sections['section'])
    else:
        latex_parse_overview[sections['section']] = {'text':[sections['text']],   'cite_spans':[sections['cite_spans']],
                                                      'cite_span_lens':[len(sections['cite_spans'])],
                                                      'section':[sections['section']], 
                                                      'bib_entries':all_articles[0]['latex_parse']}
        
    print(sections['section'],len(sections['cite_spans']))
    print(sections)
    print(10*'==')

Introduction 7
{'text': 'The goal of multi-document summarization (MDS) is to automatically generate a brief, well-organized summary for a topic which describes an event with a set of documents from different sources. BIBREF0 , BIBREF1 , BIBREF2 , BIBREF3 , BIBREF4 , BIBREF5 , BIBREF6 . In the typical setting of MDS, the input is a set of news documents about the same topic. The output summary is a piece of short text document containing several sentences, generated only based on the input original documents.', 'cite_spans': [{'start': 193, 'end': 200, 'text': None, 'latex': None, 'ref_id': 'BIBREF0'}, {'start': 203, 'end': 210, 'text': None, 'latex': None, 'ref_id': 'BIBREF1'}, {'start': 213, 'end': 220, 'text': None, 'latex': None, 'ref_id': 'BIBREF2'}, {'start': 223, 'end': 230, 'text': None, 'latex': None, 'ref_id': 'BIBREF3'}, {'start': 233, 'end': 240, 'text': None, 'latex': None, 'ref_id': 'BIBREF4'}, {'start': 243, 'end': 250, 'text': None, 'latex': None, 'ref_id': 'BIBREF5'}, 

{'text': 'VAESum BIBREF6 employs an alignment mechanism BIBREF12 , BIBREF13 to recall the lost detailed information from the input sentence. Inspired this idea, we design a jointly weighted alignment mechanism by considering the news sentence and the comment sentence simultaneously. For each decoder hidden state INLINEFORM0 , we align it with each news encoder hidden state INLINEFORM1 by an alignment vector INLINEFORM2 . We also align it with each comments encoder hidden state INLINEFORM3 by an alignment vector INLINEFORM4 . In order to filter the noisy information from the comments, we again employ the comment weight INLINEFORM5 to adjust the alignment vector of comments: DISPLAYFORM0 ', 'cite_spans': [{'start': 7, 'end': 14, 'text': None, 'latex': None, 'ref_id': 'BIBREF6'}, {'start': 46, 'end': 54, 'text': None, 'latex': None, 'ref_id': 'BIBREF12'}, {'start': 57, 'end': 65, 'text': None, 'latex': None, 'ref_id': 'BIBREF13'}], 'ref_spans': [], 'eq_spans': [{'start': 304, 'end': 315, 

{'text': 'Each topic is assigned to 4 experts, who are major in journalism, to conduct the summary writing. The task of summary writing is divided into two phases, namely, aspect facet identification, and summary generation. For the aspect facet identification, the experts read and digested all the news documents and reader comments under the topic. Then for each aspect, the experts extracted the related facets from the news document. The summaries were generated based on the annotated aspect facets. When selecting facets, one consideration is those facets that are popular in both news documents and reader comments have higher priority. Next, the facets that are popular in news documents have the next priority. The generated summary should cover as many aspects as possible, and should be well-organized using complete sentences with a length restriction of 100 words.', 'cite_spans': [], 'ref_spans': [], 'eq_spans': [], 'section': 'Data Collection'}
Data Collection 0
{'text': 'After fini

In [54]:
latex_parse_overview['Introduction']

{'text': ['The goal of multi-document summarization (MDS) is to automatically generate a brief, well-organized summary for a topic which describes an event with a set of documents from different sources. BIBREF0 , BIBREF1 , BIBREF2 , BIBREF3 , BIBREF4 , BIBREF5 , BIBREF6 . In the typical setting of MDS, the input is a set of news documents about the same topic. The output summary is a piece of short text document containing several sentences, generated only based on the input original documents.',
  "With the development of social media and mobile equipments, more and more user generated content is available. Figure FIGREF2 is a snapshot of reader comments under the news report “The most important announcements from Google's big developers' conference”. The content of the original news report talks about some new products based on AI techniques. The news report generally conveys an enthusiastic tone. However, while some readers share similar enthusiasms, some others express their worri

In [55]:
all_articles[0]['grobid_parse'].keys()

dict_keys(['abstract', 'body_text', 'ref_entries', 'bib_entries'])

##### мы должны выделить только обзорную часть из текста, а все остальноё сохранить

In [57]:
overview_papers = dict()
overview_papers[all_articles[0]['paper_id']] = {
    'paper_id':all_articles[0]['paper_id'],   'metadata':all_articles[0]['metadata'],
    's2_pdf_hash':all_articles[0]['s2_pdf_hash'], 'grobid_parse':None,'latex_parse':None}

In [58]:
overview_papers[all_articles[0]['paper_id']]['grobid_parse'] = {'abstract':all_articles[0]['grobid_parse']['abstract'],
                                                                'overview_text':grobid_parse_overview,  
                                                                'bib_entries':all_articles[0]['grobid_parse']['bib_entries']}
overview_papers[all_articles[0]['paper_id']]['latex_parse'] =  {'abstract':all_articles[0]['latex_parse']['abstract'],
                                                                'overview_text':latex_parse_overview,  
                                                                'bib_entries':all_articles[0]['latex_parse']['bib_entries']}

In [61]:
overview_papers

{'10164018': {'paper_id': '10164018',
  'metadata': {'title': 'Reader-Aware Multi-Document Summarization: An Enhanced Model and The First Dataset',
   'authors': [{'first': 'Piji', 'middle': [], 'last': 'Li', 'suffix': ''},
    {'first': 'Lidong', 'middle': [], 'last': 'Bing', 'suffix': ''},
    {'first': 'Wai', 'middle': [], 'last': 'Lam', 'suffix': ''}],
   'abstract': 'We investigate the problem of reader-aware multi-document summarization (RA-MDS) and introduce a new dataset for this problem. To tackle RA-MDS, we extend a variational auto-encodes (VAEs) based MDS framework by jointly considering news documents and reader comments. To conduct evaluation for summarization performance, we prepare a new dataset. We describe the methods for data collection, aspect annotation, and summary writing as well as scrutinizing by experts. Experimental results show that reader comments can improve the summarization performance, which also demonstrates the usefulness of the proposed dataset. The 

Для сравнения глазами ссылка на [статью](https://arxiv.org/pdf/1708.01065.pdf)

Все супер

In [62]:
for num_tex,(k,v) in enumerate(latex_parse_overview.items()):
    print(num_tex,k,sum(v['cite_span_lens']))

0 Introduction 13
1 Overview 1
2 Reader-Aware Salience Estimation 6
3 Summary Construction 6
4 Data Description 0
5 Background 0
6 Data Collection 0
7 Data Properties 0
8 Dataset and Metrics 1
9 Comparative Methods 6
10 Experimental Settings 2
11 Results on Our Dataset 0
12 Further Investigation of Our Framework  2
13 Case Study 0
14 Conclusions 0


In [63]:
for k,v in grobid_parse_overview.items():
    print(k,len(v['cite_spans']))

0 13
1 1
2 6
3 3
4 0
5 0
6 0
7 0
8 1
9 4
10 2
11 2
12 0
13 1


Упорядочим по количеству ссылок и возьмём абзац с максимальным значением, а также со 2 максимальным значением если количество ссылок в нём больше половины от максимального кол-ва 

In [64]:
grobid_parse_overview = {k: v for k, v in sorted(grobid_parse_overview.items(), 
                                                 key=lambda item: len(item[1]['cite_spans']), reverse=True)}

In [65]:
max_cite_span_sum = 0
max_grobid_parse_overview = dict()
for k,v in grobid_parse_overview.items():
    if max_cite_span_sum < len(v['cite_spans']):
        max_cite_span_sum = len(v['cite_spans'])
        max_grobid_parse_overview[k] = v
    elif (max_cite_span_sum>7) and len(v['cite_spans'])>max_cite_span_sum//2:
        max_grobid_parse_overview[k] = v

In [67]:
latex_parse_overview = {k: v for k, v in sorted(latex_parse_overview.items(), 
                                                 key=lambda item: sum(item[1]['cite_span_lens']), reverse=True)}

In [68]:
max_cite_span_sum = 0
max_latex_parse_overview = dict()
for k,v in latex_parse_overview.items():
    if max_cite_span_sum < sum(v['cite_span_lens']):
        max_cite_span_sum = sum(v['cite_span_lens'])
        max_latex_parse_overview[k] = v
    elif (max_cite_span_sum>7) and sum(v['cite_span_lens'])>max_cite_span_sum//2:
        max_latex_parse_overview[k] = v

In [70]:
for k,v in latex_parse_overview.items():
    print(k,sum(v['cite_span_lens']))

Introduction 13
Reader-Aware Salience Estimation 6
Summary Construction 6
Comparative Methods 6
Experimental Settings 2
Further Investigation of Our Framework  2
Overview 1
Dataset and Metrics 1
Data Description 0
Background 0
Data Collection 0
Data Properties 0
Results on Our Dataset 0
Case Study 0
Conclusions 0


### Применим для всех 


In [71]:
overview_papers = dict()

with_2_max= False # будем сохранять 2 максимальное значение

for num_artic,article in enumerate(all_articles):
    # проверяем что у статьи есть grobid_parse и latex_parse и естб текст
    if (article['grobid_parse'] and article['grobid_parse']['body_text']) or (article['latex_parse'] and article['latex_parse']['body_text']):
        # задаем шаблон отражения статьи в укороченном формате (чтобы занимать меньше памяти)
        overview_papers[article['paper_id']] = { 'paper_id':article['paper_id'],   'metadata':article['metadata'],
                                                 's2_pdf_hash':article['s2_pdf_hash'], 'grobid_parse':None,'latex_parse':None}
        
        grobid_parse_overview = None
        # если у статьи есть article['grobid_parse']['body_text']
        if article['grobid_parse'] and article['grobid_parse']['body_text']:
            grobid_parse_overview = dict()
            # проходим по каждому абзацу текста
            for num_sec,sections in enumerate(article['grobid_parse']['body_text']):
                grobid_parse_overview[num_sec] = sections
            
            # отсортируем по количеству цитат абзацы
            grobid_parse_overview = {k: v for k, v in sorted(grobid_parse_overview.items(), key=lambda item: len(item[1]['cite_spans']), reverse=True)}
            
            if with_2_max:
                # найдем 1 и 2 максимум по обзорной части
                max_cite_span_sum = 0
                max_grobid_parse_overview = dict()
                for k,v in grobid_parse_overview.items():
                    if max_cite_span_sum < len(v['cite_spans']):
                        max_cite_span_sum = len(v['cite_spans'])
                        max_grobid_parse_overview[k] = v
                    # записываем 2 максимум, если количество ссылок в егочасти больше половины от максимального 
                    elif (max_cite_span_sum>7) and len(v['cite_spans'])>max_cite_span_sum//2:
                        max_grobid_parse_overview[k] = v

                grobid_parse_overview = max_grobid_parse_overview
            
        latex_parse_overview = None
        # если у статьи есть article['latex_parse']['body_text']
        if article['latex_parse'] and article['latex_parse']['body_text']:
            latex_parse_overview = dict()
            # проходим по каждому абзацу текста
            # в latex_parse 
            for sections in article['latex_parse']['body_text']:
                if sections['section'] in latex_parse_overview:
                    if latex_parse_overview[sections['section']] == sections:
                        continue
                    else:
                        latex_parse_overview[sections['section']]['text'].append(sections['text'])
                        latex_parse_overview[sections['section']]['cite_spans'].append(sections['cite_spans'])
                        latex_parse_overview[sections['section']]['cite_span_lens'].append(len(sections['cite_spans']))
                        latex_parse_overview[sections['section']]['section'].append(sections['section'])
                else:
                    latex_parse_overview[sections['section']] = {'text':[sections['text']],   'cite_spans':[sections['cite_spans']],
                                                                  'cite_span_lens':[len(sections['cite_spans'])],
                                                                  'section':[sections['section']]}
            latex_parse_overview = {k: v for k, v in sorted(latex_parse_overview.items(), key=lambda item: item[1]['cite_span_lens'], reverse=True)}
        
            if with_2_max:
                max_cite_span_sum = 0
                max_latex_parse_overview = dict()
                for k,v in latex_parse_overview.items():
                    if max_cite_span_sum < sum(v['cite_span_lens']):
                        max_cite_span_sum = sum(v['cite_span_lens'])
                        max_latex_parse_overview[k] = v
                    elif (max_cite_span_sum>0) and sum(v['cite_span_lens'])>max_cite_span_sum//2:
                        max_latex_parse_overview[k] = v

                latex_parse_overview = max_latex_parse_overview
        

        if grobid_parse_overview:
            overview_papers[article['paper_id']]['grobid_parse'] = {'abstract':None,
                                                        'overview_text':grobid_parse_overview,  
                                                        'bib_entries':None}
            if article['grobid_parse']['abstract']:
                overview_papers[article['paper_id']]['grobid_parse']['abstract'] = article['grobid_parse']['abstract']
            if article['grobid_parse']['bib_entries']:
                overview_papers[article['paper_id']]['grobid_parse']['bib_entries'] = article['grobid_parse']['bib_entries']
            
        if latex_parse_overview:            
            overview_papers[article['paper_id']]['latex_parse'] = {'abstract':None,
                                                                    'overview_text':latex_parse_overview,  
                                                                    'bib_entries':None}
            if article['latex_parse']['abstract']:
                overview_papers[article['paper_id']]['latex_parse']['abstract'] = article['latex_parse']['abstract']
            if article['latex_parse']['bib_entries']:
                overview_papers[article['paper_id']]['latex_parse']['bib_entries'] = article['latex_parse']['bib_entries']

In [72]:
list(overview_papers.keys())[:10]

['10164018',
 '14472576',
 '17302615',
 '3243536',
 '3248240',
 '2223737',
 '488',
 '14323173',
 '15251605',
 '8260435']

##### Выделим стать, у которых есть latex_parse

In [76]:
overview_papers_w_latex = {k:v for k,v in overview_papers.items() if v['latex_parse']}

In [77]:
len(overview_papers_w_latex)

4039

In [410]:
def mult_in(list_toks,line):
    flag = False
    line = line.lower()
    for tok in list_toks:
        if tok.lower() in line:
            flag = tok
            break
    return flag

# Расчет покрытия

## Покрытие по всему тексту статьи

In [79]:
with open('all_in_paper.txt', encoding="utf8") as fin:
    content = fin.readlines()
fin.close()

In [80]:
all_dict_in_ids = [x.strip() for x in content]
del content

In [81]:
dict_in_ids_all = {val:1 for val in all_dict_in_ids}
len(dict_in_ids_all)

5986243

In [82]:
all_articles[0].keys()

dict_keys(['paper_id', 'metadata', 's2_pdf_hash', 'grobid_parse', 'latex_parse'])

In [83]:
all_articles[0]['metadata'].keys()

dict_keys(['title', 'authors', 'abstract', 'year', 'arxiv_id', 'acl_id', 'pmc_id', 'pubmed_id', 'doi', 'venue', 'journal'])

In [84]:
all_articles[0]['grobid_parse'].keys()

dict_keys(['abstract', 'body_text', 'ref_entries', 'bib_entries'])

In [125]:
bib_entries_grobid = dict()
for k,v in all_articles[0]['grobid_parse']['bib_entries'].items():
    print(k)
    print(v['title'])
    print(v['links'])
    bib_entries_grobid[v['title']] = v['links']

BIBREF0
Neural machine translation by jointly learning to align and translate
11212020
BIBREF1
Theano: new features and speed improvements
8180128
BIBREF2
Abstractive multidocument summarization via phrase selection and merging
8377315
BIBREF3
Linear programming 1: introduction
53739754
BIBREF4
Lexpagerank: Prestige in multi-document text summarization
10418456
BIBREF5
A virtual reality headset that's coming to the PlayStation 4. Sony showed off a prototype device V called Project Morpheus V that can be worn to create a virtual reality experience when playing games on its new PlayStation 4 console. Mr. Yoshida said that Sony was inspired and encouraged to do its own virtual reality project after the enthusiastic response to the efforts of Oculus VR and Valve
None
BIBREF6
Multi-document summarization by sentence extraction
8294822
BIBREF7
Comments-oriented document summarization: Understanding documents with readers' feedback
13723748
BIBREF8
Adam: A method for stochastic optimization
6

In [126]:
bib_entries_latex = dict()
for k,v in all_articles[0]['latex_parse']['bib_entries'].items():
    print(k)
    print(v['title'],v['links'])
    bib_entries_latex[v['title']] = v['links']

BIBREF5
Abstractive multi-document summarization via phrase selection and merging 8377315
BIBREF15
Linear programming 1: introduction 53739754
BIBREF1
Lexpagerank: Prestige in multi-document text summarization 10418456
BIBREF0
Multi-document summarization by sentence extraction 8294822
BIBREF7
Comments-oriented document summarization: Understanding documents with readers' feedback 13723748
BIBREF20
Adam: A method for stochastic optimization 6628106
BIBREF10
Auto-encoding variational bayes 15789289
BIBREF9
Reader-aware multi-document summarization via sparse coding 14777460
BIBREF6
Salience estimation via variational auto-encoders for multi-document summarization 29562039
BIBREF13
Effective approaches to attention-based neural machine translation 1998416
BIBREF19
Textrank: Bringing order into texts 577937
BIBREF4
Exploiting category-specific information for multi-document summarization 6317274
BIBREF3
A survey of text summarization techniques 556431
BIBREF18
Centroid-based summarization

In [127]:
bib_entries_grobid = {k: v for k, v in sorted(bib_entries_grobid.items(), key=lambda item: item[0])}


In [128]:
bib_entries_latex = {k: v for k, v in sorted(bib_entries_latex.items(), key=lambda item: item[0])}


In [129]:
all_bib_entries = dict()
if bib_entries_latex:
    for k,v in bib_entries_latex.items():
        all_bib_entries[k] = [v]
if bib_entries_grobid:
    for k,v in bib_entries_grobid.items():
        if k in all_bib_entries:
            if all_bib_entries[k][0] == v:
                continue
            else:
                all_bib_entries[k].append(v)
        else:
            all_bib_entries[k] = [v]

In [130]:
len(all_bib_entries),len(bib_entries_grobid),len(bib_entries_latex)

(28, 23, 18)

In [131]:
all_articles[1]

{'paper_id': '14472576',
 'metadata': {'title': 'Building a Semantic Parser Overnight',
  'authors': [{'first': 'Yushi', 'middle': [], 'last': 'Wang', 'suffix': ''},
   {'first': 'Jonathan', 'middle': [], 'last': 'Berant', 'suffix': ''},
   {'first': 'Percy', 'middle': [], 'last': 'Liang', 'suffix': ''}],
  'abstract': 'How do we build a semantic parser in a new domain starting with zero training examples? We introduce a new methodology for this setting: First, we use a simple grammar to generate logical forms paired with canonical utterances. The logical forms are meant to cover the desired set of compositional operators, and the canonical utterances are meant to capture the meaning of the logical forms (although clumsily). We then use crowdsourcing to paraphrase these canonical utterances into natural utterances. The resulting data is used to train the semantic parser. We further study the role of compositionality in the resulting paraphrases. Finally, we test our methodology on seve

In [132]:
all_bib_entries

{'A survey of text summarization techniques': ['556431'],
 'Abstractive multi-document summarization via phrase selection and merging': ['8377315'],
 'Adam: A method for stochastic optimization': ['6628106'],
 'Auto-encoding variational bayes': ['15789289'],
 'Centroid-based summarization of multiple documents: sentence extraction, utility-based evaluation, and user studies': ['1320'],
 "Comments-oriented document summarization: Understanding documents with readers' feedback": ['13723748'],
 'Effective approaches to attention-based neural machine translation': ['1998416'],
 'Exploiting category-specific information for multi-document summarization': ['6317274'],
 'Lexpagerank: Prestige in multi-document text summarization': ['10418456'],
 'Linear programming 1: introduction': ['53739754'],
 'Multi-document summarization by sentence extraction': ['8294822'],
 'Multiple aspect summarization using integer linear programming': ['17497992'],
 'Reader-aware multi-document summarization via s

In [194]:
article_bibs = {}
for num_artic,article in enumerate(all_articles):
#     if num_artic >2:
#         break
    
    # проверка наличия определленных значений статьи
    if not ((article['grobid_parse'] and article['grobid_parse']['bib_entries']) or (article['latex_parse'] and article['latex_parse']['body_text'])):
#         article_bibs[article['paper_id']] = 'None'
        continue
        
    bib_entries_grobid = None
    if article['grobid_parse'] and article['grobid_parse']['bib_entries']:
        bib_entries_grobid = dict()
        
        for k,v in article['grobid_parse']['bib_entries'].items():
            bib_entries_grobid[v['title']] = v['links']
            
        bib_entries_grobid = {k: v for k, v in sorted(bib_entries_grobid.items(), key=lambda item: item[0])}
        
    bib_entries_latex = None
    if article['latex_parse'] and article['latex_parse']['body_text']:
        bib_entries_latex = dict()
        for k,v in article['latex_parse']['bib_entries'].items():
            bib_entries_latex[v['title']] = v['links']
            
        bib_entries_latex = {k: v for k, v in sorted(bib_entries_latex.items(), key=lambda item: item[0])}
    
    all_bib_entries = dict()
    if bib_entries_latex:
        for k,v in bib_entries_latex.items():
            if k == '':
                continue
                
            if v:
                all_bib_entries[k] = [v]
            else:
                all_bib_entries[k] = []

    if bib_entries_grobid:
        for k,v in bib_entries_grobid.items():
            if k == '':
                continue
                
            if k in all_bib_entries:
                if len(all_bib_entries[k]) == 0:
                    if v:
                        all_bib_entries[k] = [v]
                    else:
                        all_bib_entries[k] = []
                else:
                    if all_bib_entries[k][0] == v:
                        continue
                    else:
                        if v:
                            all_bib_entries[k].append(v)
                        else:
                            continue
            else:
                if v:
                    all_bib_entries[k] = [v]
                else:
                    all_bib_entries[k] = []
    article_bibs[article['paper_id']] = all_bib_entries

In [195]:
len(article_bibs)

39856

In [159]:
len(article_bibs['10164018']),article_bibs['10164018']

(28,
 {'A survey of text summarization techniques': ['556431'],
  'Abstractive multi-document summarization via phrase selection and merging': ['8377315'],
  'Adam: A method for stochastic optimization': ['6628106'],
  'Auto-encoding variational bayes': ['15789289'],
  'Centroid-based summarization of multiple documents: sentence extraction, utility-based evaluation, and user studies': ['1320'],
  "Comments-oriented document summarization: Understanding documents with readers' feedback": ['13723748'],
  'Effective approaches to attention-based neural machine translation': ['1998416'],
  'Exploiting category-specific information for multi-document summarization': ['6317274'],
  'Lexpagerank: Prestige in multi-document text summarization': ['10418456'],
  'Linear programming 1: introduction': ['53739754'],
  'Multi-document summarization by sentence extraction': ['8294822'],
  'Multiple aspect summarization using integer linear programming': ['17497992'],
  'Reader-aware multi-document s

In [160]:
len(article_bibs['14472576']),article_bibs['14472576']

(27,
 {'A joint model of language and perception for grounded attribute learning': ['2408319'],
  'Adaptive subgradient methods for online learning and stochastic optimization': ['538820'],
  'Alignment by agreement': ['618683'],
  'An efficient easily adaptable system for interpreting natural language queries': ['2498523'],
  'Bootstrapping semantic parsers from conversations': ['1140108'],
  'Compositional semantic parsing on semi-structured tables': ['9027681'],
  'Controlled natural languages for knowledge representation': ['10228634'],
  "Driving semantic parsing from the world's response": ['5667590'],
  'Features and pitfalls that users should seek in natural language interfaces to databases': [],
  'Jointly learning to parse and perceive: Connecting natural language to the physical world': ['10250712'],
  'Lambda dependency-based compositional semantics. arXiv': [],
  'Large-scale semantic parsing via schema matching and lexicon extension': ['2265838'],
  'Largescale semantic p

In [198]:
index = []
num_all_bibs = []
num_all_bibs_with_links = []
num_all_bibs_w_l_in_dict_in = []
for num_artic,(key,val_dict) in enumerate(article_bibs.items()):
    index.append(key)
    num_bibs = 0
    num_bibs_with_links = 0
    num_bibs_w_l_in_dict_in = 0
    try:
        for k,val in val_dict.items():
            num_bibs +=1
            if len(val)>0:
                num_bibs_with_links+=1
            for link in val:
                if link in dict_in_ids_all:
                    num_bibs_w_l_in_dict_in+=1
                    break
    except:
        print(num_artic)
        print(key)
        break
    num_all_bibs.append(num_bibs)
    num_all_bibs_with_links.append(num_bibs_with_links)
    num_all_bibs_w_l_in_dict_in.append(num_bibs_w_l_in_dict_in)

In [203]:
d = {'#bibs':num_all_bibs,'#bibs_with_links':num_all_bibs_with_links,'#bibs_with_links_in_dict':num_all_bibs_w_l_in_dict_in}
df_covering =  pd.DataFrame(d)
df_covering.index = index

In [206]:
df_covering['%_in_dict2bibs'] = df_covering['#bibs_with_links_in_dict']/df_covering['#bibs']
df_covering['%_in_dict2bibs_w_l'] = df_covering['#bibs_with_links_in_dict']/df_covering['#bibs_with_links']

Пояснение к колонкам:
 - **#bibs** -  количество всех распознанных Reference статей
 - **#bibs_with_links** - количество всех распознанных Reference статей, которые присутствуют в датасете GORC
 - **#bibs_with_links_in_dict** - количество всех распознанных Reference статей, которые присутствуют в структуре данных ***in***,где есть статья и ссылающиеся на неё статьи
 - **%_in_dict2bibs** - отношение кол-ва ref **in** ко всему кол-ву ref статьи 
 - **%_in_dict2bibs_w_l** - отношение кол-ва ref **in** ко всему кол-ву ref статьей, которые присутствуют в датасете GORC

In [216]:
df_covering.head()

,#bibs,#bibs_with_links,#bibs_with_links_in_dict,%_in_dict2bibs,%_in_dict2bibs_w_l
10164018,28,26,20,0.714286,0.769231
14472576,27,25,21,0.777778,0.840000
17302615,8,5,4,0.500000,0.800000
3243536,10,3,2,0.200000,0.666667
3248240,16,12,8,0.500000,0.666667


In [217]:
df_covering['%_in_dict2bibs'].mean()

0.38197519738407737

In [218]:
df_covering['%_in_dict2bibs_w_l'].mean()

0.5416087540264509

## Покрытие по обзорным частям статьи

In [221]:
overview_papers = dict()
for num_artic,article in enumerate(all_articles):
    # проверяем что у статьи есть grobid_parse и latex_parse и естб текст
    if (article['grobid_parse'] and article['grobid_parse']['body_text']) or (article['latex_parse'] and article['latex_parse']['body_text']):
        # задаем шаблон отражения статьи в укороченном формате (чтобы занимать меньше памяти)
        overview_papers[article['paper_id']] = { 'paper_id':article['paper_id'],   'metadata':article['metadata'],
                                                 's2_pdf_hash':article['s2_pdf_hash'], 'grobid_parse':None,'latex_parse':None}
        
        grobid_parse_overview = None
        # если у статьи есть article['grobid_parse']['body_text']
        if article['grobid_parse'] and article['grobid_parse']['body_text']:
            grobid_parse_overview = dict()
            # проходим по каждому абзацу текста
            for num_sec,sections in enumerate(article['grobid_parse']['body_text']):
                grobid_parse_overview[num_sec] = sections
            
            # отсортируем по количеству цитат абзацы
            grobid_parse_overview = {k: v for k, v in sorted(grobid_parse_overview.items(), key=lambda item: len(item[1]['cite_spans']), reverse=True)}
            
#             # найдем 1 , пока без 2 максимума по обзорной части
            max_cite_span_sum = 0
            max_grobid_parse_overview = dict()
            for k,v in grobid_parse_overview.items():
                if max_cite_span_sum < len(v['cite_spans']):
                    max_cite_span_sum = len(v['cite_spans'])
                    max_grobid_parse_overview[k] = v
                    
#                 # записываем 2 максимум, если количество ссылок в егочасти больше половины от максимального 
#                 elif (max_cite_span_sum>7) and len(v['cite_spans'])>max_cite_span_sum//2:
#                     max_grobid_parse_overview[k] = v
                    
            
            grobid_parse_overview = max_grobid_parse_overview
            
        latex_parse_overview = None
        # если у статьи есть article['latex_parse']['body_text']
        if article['latex_parse'] and article['latex_parse']['body_text']:
            latex_parse_overview = dict()
            # проходим по каждому абзацу текста
            # в latex_parse 
            for sections in article['latex_parse']['body_text']:
                if sections['section'] in latex_parse_overview:
                    if latex_parse_overview[sections['section']] == sections:
                        continue
                    else:
                        latex_parse_overview[sections['section']]['text'].append(sections['text'])
                        latex_parse_overview[sections['section']]['cite_spans'].append(sections['cite_spans'])
                        latex_parse_overview[sections['section']]['cite_span_lens'].append(len(sections['cite_spans']))
                        latex_parse_overview[sections['section']]['section'].append(sections['section'])
                else:
                    latex_parse_overview[sections['section']] = {'text':[sections['text']],   'cite_spans':[sections['cite_spans']],
                                                                  'cite_span_lens':[len(sections['cite_spans'])],
                                                                  'section':[sections['section']]}
            latex_parse_overview = {k: v for k, v in sorted(latex_parse_overview.items(), key=lambda item: item[1]['cite_span_lens'], reverse=True)}
        
        
            max_cite_span_sum = 0
            max_latex_parse_overview = dict()
            for k,v in latex_parse_overview.items():
                if max_cite_span_sum < sum(v['cite_span_lens']):
                    max_cite_span_sum = sum(v['cite_span_lens'])
                    max_latex_parse_overview[k] = v
#                 elif (max_cite_span_sum>0) and sum(v['cite_span_lens'])>max_cite_span_sum//2:
#                     max_latex_parse_overview[k] = v
            
            latex_parse_overview = max_latex_parse_overview
        

        if grobid_parse_overview:
            overview_papers[article['paper_id']]['grobid_parse'] = {'abstract':None,
                                                        'overview_text':grobid_parse_overview,  
                                                        'bib_entries':None}
            if article['grobid_parse']['abstract']:
                overview_papers[article['paper_id']]['grobid_parse']['abstract'] = article['grobid_parse']['abstract']
            if article['grobid_parse']['bib_entries']:
                overview_papers[article['paper_id']]['grobid_parse']['bib_entries'] = article['grobid_parse']['bib_entries']
            
        if latex_parse_overview:            
            overview_papers[article['paper_id']]['latex_parse'] = {'abstract':None,
                                                                    'overview_text':latex_parse_overview,  
                                                                    'bib_entries':None}
            if article['latex_parse']['abstract']:
                overview_papers[article['paper_id']]['latex_parse']['abstract'] = article['latex_parse']['abstract']
            if article['latex_parse']['bib_entries']:
                overview_papers[article['paper_id']]['latex_parse']['bib_entries'] = article['latex_parse']['bib_entries']

In [222]:
list(overview_papers)[:10]

['10164018',
 '14472576',
 '17302615',
 '3243536',
 '3248240',
 '2223737',
 '488',
 '14323173',
 '15251605',
 '8260435']

In [282]:
print(overview_papers['10164018'].keys())
print(overview_papers['10164018']['grobid_parse'].keys())
print(overview_papers['10164018']['latex_parse'].keys())
print(overview_papers['10164018']['grobid_parse']['overview_text'][0].keys())
print(overview_papers['488']['latex_parse']['overview_text']['Introduction'].keys())

dict_keys(['paper_id', 'metadata', 's2_pdf_hash', 'grobid_parse', 'latex_parse'])
dict_keys(['abstract', 'overview_text', 'bib_entries'])
dict_keys(['abstract', 'overview_text', 'bib_entries'])
dict_keys(['text', 'cite_spans', 'ref_spans', 'eq_spans', 'section'])
dict_keys(['text', 'cite_spans', 'cite_span_lens', 'section'])


In [327]:
overview_papers['189927790']['grobid_parse']['overview_text'][0]['cite_spans']

[{'start': 99,
  'end': 110,
  'text': '(Nie, 2010)',
  'latex': None,
  'ref_id': 'BIBREF7'},
 {'start': 223,
  'end': 245,
  'text': '(Sokokov et al., 2013)',
  'latex': None,
  'ref_id': 'BIBREF9'},
 {'start': 513,
  'end': 535,
  'text': '(Sokolov et al., 2014)',
  'latex': None,
  'ref_id': 'BIBREF10'},
 {'start': 717,
  'end': 738,
  'text': '(Sokokov et al., 2013',
  'latex': None,
  'ref_id': 'BIBREF9'},
 {'start': 1662,
  'end': 1681,
  'text': '(Zhou et al., 2012)',
  'latex': None,
  'ref_id': None},
 {'start': 1933,
  'end': 1956,
  'text': '(Bahdanau et al., 2015)',
  'latex': None,
  'ref_id': 'BIBREF0'},
 {'start': 2195,
  'end': 2219,
  'text': '(Zamani and Croft, 2017)',
  'latex': None,
  'ref_id': 'BIBREF13'},
 {'start': 2265,
  'end': 2287,
  'text': '(Mikolov et al., 2013)',
  'latex': None,
  'ref_id': 'BIBREF6'},
 {'start': 2491,
  'end': 2511,
  'text': '(Luong et al., 2015)',
  'latex': None,
  'ref_id': 'BIBREF5'}]

In [326]:
overview_papers['189927790']['grobid_parse']['bib_entries']

{'BIBREF0': {'ref_id': 'b0',
  'title': 'Neural machine translation by jointly learning to align and translate',
  'authors': [{'first': 'Dzmitry',
    'middle': [],
    'last': 'Bahdanau',
    'suffix': ''},
   {'first': 'Kyunghyun', 'middle': [], 'last': 'Cho', 'suffix': ''},
   {'first': 'Yoshua', 'middle': [], 'last': 'Bengio', 'suffix': ''}],
  'year': 2015,
  'venue': 'International Conference on Learning Representations (ICLR)',
  'volume': '',
  'issn': '',
  'pages': '',
  'other_ids': {},
  'links': '11212020'},
 'BIBREF1': {'ref_id': 'b1',
  'title': 'NLTK: The natural language toolkit',
  'authors': [{'first': 'Steven', 'middle': [], 'last': 'Bird', 'suffix': ''},
   {'first': 'Edward', 'middle': [], 'last': 'Loper', 'suffix': ''}],
  'year': 2004,
  'venue': 'Proceedings of the ACL Interactive Poster and Demonstration Sessions',
  'volume': '',
  'issn': '',
  'pages': '',
  'other_ids': {},
  'links': '1438450'},
 'BIBREF2': {'ref_id': 'b2',
  'title': 'Training neural ma

In [325]:
bib_entries_grobid = dict()

paper = overview_papers['189927790']
key_grobid = list(paper['grobid_parse']['overview_text'].keys())[0]
for cite_span in paper['grobid_parse']['overview_text'][key_grobid]['cite_spans']:
    cite_ref = cite_span['ref_id']
    cited_paper_id = None
    if cite_ref in paper['grobid_parse']['bib_entries']:
        cited_paper_id = paper['grobid_parse']['bib_entries'][cite_ref]['links']
    if cited_paper_id:
        print(cite_ref,cited_paper_id,paper['grobid_parse']['bib_entries'][cite_ref]['title'])
        bib_entries_grobid[paper['grobid_parse']['bib_entries'][cite_ref]['title']] = cited_paper_id
    elif cite_ref  in paper['grobid_parse']['bib_entries']:
        print(cite_ref,cited_paper_id,paper['grobid_parse']['bib_entries'][cite_ref]['title'])
        bib_entries_grobid[paper['grobid_parse']['bib_entries'][cite_ref]['title']] = cited_paper_id
    else:
        # неверно распознается, это редко статья
        print('kek',cite_ref,cited_paper_id)
    

BIBREF7 None Cross-Language Information Retrieval. Synthesis Lectures on Human Language Technologies
BIBREF9 2790246 Boosting cross-language retrieval by learning bilingual phrase associations from relevance rankings
BIBREF10 46107 Learning to translate queries for CLIR
BIBREF9 2790246 Boosting cross-language retrieval by learning bilingual phrase associations from relevance rankings
kek None None
BIBREF0 11212020 Neural machine translation by jointly learning to align and translate
BIBREF13 None Relevancebased word embedding
BIBREF6 16447573 Distributed representations of words and phrases and their compositionality
BIBREF5 1998416 Effective approaches to attention-based neural machine translation


In [305]:
paper['grobid_parse']['overview_text'].keys()

dict_keys([3])

In [265]:
bib_entries_grobid

{'Multi-document summarization by sentence extraction': '8294822',
 'Lexpagerank: Prestige in multi-document text summarization': '10418456',
 'Manifold-ranking based topic-focused multidocument summarization': '532313',
 'A survey of text summarization techniques': '556431',
 'Exploiting category-specific information for multidocument summarization': '6317274',
 'Salience estimation via variational auto-encoders for multi-document summarization': '29562039',
 "Comments-oriented document summarization: Understanding documents with readers' feedback": '13723748',
 'Social context summarization': '704517',
 'Abstractive multidocument summarization via phrase selection and merging': '8377315',
 'Autoencoding variational bayes': None,
 'Stochastic backpropagation and approximate inference in deep generative models': '16895865'}

In [267]:
bib_entries_grobid = {k: v for k, v in sorted(bib_entries_grobid.items(), key=lambda item: item[0])}
bib_entries_grobid

{'A survey of text summarization techniques': '556431',
 'Abstractive multidocument summarization via phrase selection and merging': '8377315',
 'Autoencoding variational bayes': None,
 "Comments-oriented document summarization: Understanding documents with readers' feedback": '13723748',
 'Exploiting category-specific information for multidocument summarization': '6317274',
 'Lexpagerank: Prestige in multi-document text summarization': '10418456',
 'Manifold-ranking based topic-focused multidocument summarization': '532313',
 'Multi-document summarization by sentence extraction': '8294822',
 'Salience estimation via variational auto-encoders for multi-document summarization': '29562039',
 'Social context summarization': '704517',
 'Stochastic backpropagation and approximate inference in deep generative models': '16895865'}

In [328]:
overview_papers['189927790']['latex_parse']

{'abstract': None,
 'overview_text': {'Introduction': {'text': ['CLIR systems retrieve documents written in a language that is different from search query language BIBREF0 . The primary objective of CLIR is to translate or project a query into the language of the document repository BIBREF1 , which we refer to as Retrieval Corpus (RC). To this end, common CLIR approaches translate search queries using a Machine Translation (MT) model and then use a monolingual IR system to retrieve from RC. In this process, a translation model is treated as a black box BIBREF2 , and it is usually trained on a sentence level parallel corpus, which we refer to as Translation Corpus (TC).',
    'We address a pitfall of using existing MT models for query translation BIBREF1 . An MT model trained on TC does not have any knowledge of RC. In an extreme setting, where there are no common terms between the target side of TC and RC, a well trained and tested translation model would fail because of vocabulary mis

In [335]:
all_articles[37]['latex_parse']['bib_entries']

{}

In [332]:
overview_papers['189927790']['latex_parse']['overview_text']['Introduction']['cite_spans']

[[{'start': 99, 'end': 106, 'text': None, 'latex': None, 'ref_id': 'BIBREF0'},
  {'start': 219, 'end': 226, 'text': None, 'latex': None, 'ref_id': 'BIBREF1'},
  {'start': 494,
   'end': 501,
   'text': None,
   'latex': None,
   'ref_id': 'BIBREF2'}],
 [{'start': 71, 'end': 78, 'text': None, 'latex': None, 'ref_id': 'BIBREF1'}],
 [{'start': 235, 'end': 242, 'text': None, 'latex': None, 'ref_id': 'BIBREF3'},
  {'start': 494, 'end': 501, 'text': None, 'latex': None, 'ref_id': 'BIBREF4'},
  {'start': 740, 'end': 747, 'text': None, 'latex': None, 'ref_id': 'BIBREF5'},
  {'start': 793,
   'end': 800,
   'text': None,
   'latex': None,
   'ref_id': 'BIBREF6'}],
 [{'start': 122,
   'end': 129,
   'text': None,
   'latex': None,
   'ref_id': 'BIBREF7'}]]

In [331]:
{k: v for k, v in sorted(overview_papers['10164018']['latex_parse']['bib_entries'].items(), key=lambda item: item[0])}

{'BIBREF0': {'ref_id': 'BIBREF0',
  'title': 'Multi-document summarization by sentence extraction',
  'authors': [{'first': 'Jade',
    'middle': [],
    'last': 'Goldstein',
    'suffix': ''},
   {'first': 'Vibhu', 'middle': [], 'last': 'Mittal', 'suffix': ''},
   {'first': 'Jaime', 'middle': [], 'last': 'Carbonell', 'suffix': ''},
   {'first': 'Mark', 'middle': [], 'last': 'Kantrowitz', 'suffix': ''}],
  'year': 2000,
  'venue': 'NAACL-ANLPWorkshop',
  'volume': '',
  'issn': '',
  'pages': '40--48',
  'other_ids': {},
  'links': '8294822'},
 'BIBREF1': {'ref_id': 'BIBREF1',
  'title': 'Lexpagerank: Prestige in multi-document text summarization',
  'authors': [{'first': 'Günes', 'middle': [], 'last': 'Erkan', 'suffix': ''},
   {'first': '', 'middle': [], 'last': 'Dragomir R Radev', 'suffix': ''}],
  'year': 2004,
  'venue': 'EMNLP',
  'volume': '4',
  'issn': '',
  'pages': '365--371',
  'other_ids': {},
  'links': '10418456'},
 'BIBREF10': {'ref_id': 'BIBREF10',
  'title': 'Auto-enc

In [329]:
article = overview_papers['189927790']

bib_entries_latex = dict()
for cite_spans in article['latex_parse']['overview_text']['Introduction']['cite_spans']:
    if len(cite_spans)>1:
        for cite_span in cite_spans:
            cite_ref = cite_span['ref_id']
            cited_article_id = None
            if cite_ref in article['latex_parse']['bib_entries']:
                cited_article_id = article['latex_parse']['bib_entries'][cite_ref]['links']
                
            if cited_article_id:
                print(cite_ref,cited_article_id,article['latex_parse']['bib_entries'][cite_ref]['title'])
                bib_entries_latex[article['latex_parse']['bib_entries'][cite_ref]['title']] = cited_article_id
            elif cite_ref in article['latex_parse']['bib_entries']:
                print(cite_ref,cited_article_id)
                
                bib_entries_latex[article['latex_parse']['bib_entries'][cite_ref]['title']] = cited_article_id
            else:
                print('kek',cite_ref,cited_article_id)

TypeError: argument of type 'NoneType' is not iterable

In [278]:
all_bib_entries = dict()
if bib_entries_latex:
    for k,v in bib_entries_latex.items():
        all_bib_entries[k] = [v]
if bib_entries_grobid:
    for k,v in bib_entries_grobid.items():
        if k in all_bib_entries:
            if all_bib_entries[k][0] == v:
                continue
            else:
                all_bib_entries[k].append(v)
        else:
            all_bib_entries[k] = [v]

In [279]:
all_bib_entries

{'Multi-document summarization by sentence extraction': ['8294822'],
 'Lexpagerank: Prestige in multi-document text summarization': ['10418456'],
 'A survey of text summarization techniques': ['556431'],
 'Exploiting category-specific information for multi-document summarization': ['6317274'],
 'Abstractive multi-document summarization via phrase selection and merging': ['8377315'],
 'Salience estimation via variational auto-encoders for multi-document summarization': ['29562039'],
 "Comments-oriented document summarization: Understanding documents with readers' feedback": ['13723748'],
 'Social context summarization': ['704517'],
 'Reader-aware multi-document summarization via sparse coding': ['14777460'],
 'Auto-encoding variational bayes': ['15789289'],
 'Stochastic backpropagation and approximate inference in deep generative models': ['16895865'],
 'Manifold-ranking based topic-focused multidocument summarization': ['532313'],
 'Exploiting category-specific information for multidoc

In [344]:
article_bibs = {}
for num_artic,article in enumerate(overview_papers):
#     if num_artic >10:
#         break
#     print(num_artic,article)
    article = overview_papers[article]
    # проверка наличия определленных значений статьи
    if not ((article['grobid_parse'] and article['grobid_parse']['bib_entries']) or (article['latex_parse'] and article['latex_parse']['bib_entries'])):
#         article_bibs[article['paper_id']] = 'None'
        continue
        
    bib_entries_grobid = None
    if article['grobid_parse'] and article['grobid_parse']['bib_entries'] and article['grobid_parse']['overview_text']:
        bib_entries_grobid = dict()
        
        key_grobid = list(article['grobid_parse']['overview_text'].keys())[0]
        for cite_span in article['grobid_parse']['overview_text'][key_grobid]['cite_spans']:
            cite_ref = cite_span['ref_id']
            cited_paper_id = None
            if cite_ref in article['grobid_parse']['bib_entries']:
                cited_paper_id = article['grobid_parse']['bib_entries'][cite_ref]['links']
            if cited_paper_id:
                bib_entries_grobid[article['grobid_parse']['bib_entries'][cite_ref]['title']] = cited_paper_id
            elif cite_ref  in article['grobid_parse']['bib_entries']:
                bib_entries_grobid[article['grobid_parse']['bib_entries'][cite_ref]['title']] = cited_paper_id
        
            
        bib_entries_grobid = {k: v for k, v in sorted(bib_entries_grobid.items(), key=lambda item: item[0])}
        
    bib_entries_latex = None
    if article['latex_parse'] and article['latex_parse']['overview_text'] and article['latex_parse']['bib_entries']:
        bib_entries_latex = dict()
        
        key_tex = list(article['latex_parse']['overview_text'].keys())[0]
        for cite_spans in article['latex_parse']['overview_text'][key_tex]['cite_spans']:
            if len(cite_spans)>1:
                for cite_span in cite_spans:
                    cite_ref = cite_span['ref_id']
                    cited_article_id = None
                    if cite_ref in article['latex_parse']['bib_entries']:
                        cited_article_id = article['latex_parse']['bib_entries'][cite_ref]['links']
                    if cited_article_id:
                        bib_entries_latex[article['latex_parse']['bib_entries'][cite_ref]['title']] = cited_article_id
                    elif cite_ref in article['latex_parse']['bib_entries']:
                        bib_entries_latex[article['latex_parse']['bib_entries'][cite_ref]['title']] = cited_article_id
    
        bib_entries_latex = {k: v for k, v in sorted(bib_entries_latex.items(), key=lambda item: item[0])}
    
    all_bib_entries = dict()
    if bib_entries_latex:
        for k,v in bib_entries_latex.items():
            if k == '':
                continue
                
            if v:
                all_bib_entries[k] = [v]
            else:
                all_bib_entries[k] = []

    if bib_entries_grobid:
        for k,v in bib_entries_grobid.items():
            if k == '':
                continue
                
            if k in all_bib_entries:
                if len(all_bib_entries[k]) == 0:
                    if v:
                        all_bib_entries[k] = [v]
                    else:
                        all_bib_entries[k] = []
                else:
                    if all_bib_entries[k][0] == v:
                        continue
                    else:
                        if v:
                            all_bib_entries[k].append(v)
                        else:
                            continue
            else:
                if v:
                    all_bib_entries[k] = [v]
                else:
                    all_bib_entries[k] = []
    article_bibs[article['paper_id']] = all_bib_entries

In [345]:
len(article_bibs)

39428

In [346]:
index = []
num_all_bibs = []
num_all_bibs_with_links = []
num_all_bibs_w_l_in_dict_in = []
for num_artic,(key,val_dict) in enumerate(article_bibs.items()):
    index.append(key)
    num_bibs = 0
    num_bibs_with_links = 0
    num_bibs_w_l_in_dict_in = 0
    try:
        for k,val in val_dict.items():
            num_bibs +=1
            if len(val)>0:
                num_bibs_with_links+=1
            for link in val:
                if link in dict_in_ids_all:
                    num_bibs_w_l_in_dict_in+=1
                    break
    except:
        print(num_artic)
        print(key)
        break
    num_all_bibs.append(num_bibs)
    num_all_bibs_with_links.append(num_bibs_with_links)
    num_all_bibs_w_l_in_dict_in.append(num_bibs_w_l_in_dict_in)

In [348]:
d_overview = {'#bibs':num_all_bibs,'#bibs_with_links':num_all_bibs_with_links,'#bibs_with_links_in_dict':num_all_bibs_w_l_in_dict_in}
df_covering_overview =  pd.DataFrame(d)
df_covering_overview.index = index

In [349]:
df_covering_overview['%_in_dict2bibs'] = df_covering_overview['#bibs_with_links_in_dict']/df_covering_overview['#bibs']
df_covering_overview['%_in_dict2bibs_w_l'] = df_covering_overview['#bibs_with_links_in_dict']/df_covering_overview['#bibs_with_links']

Пояснение к колонкам:
 - **#bibs** -  количество всех распознанных Reference статей
 - **#bibs_with_links** - количество всех распознанных Reference статей, которые присутствуют в датасете GORC
 - **#bibs_with_links_in_dict** - количество всех распознанных Reference статей, которые присутствуют в структуре данных ***in***,где есть статья и ссылающиеся на неё статьи
 - **%_in_dict2bibs** - отношение кол-ва ref **in** ко всему кол-ву ref статьи 
 - **%_in_dict2bibs_w_l** - отношение кол-ва ref **in** ко всему кол-ву ref статьей, которые присутствуют в датасете GORC

In [351]:
df_covering_overview.head()

,#bibs,#bibs_with_links,#bibs_with_links_in_dict,%_in_dict2bibs,%_in_dict2bibs_w_l
10164018,15,14,9,0.600000,0.642857
14472576,18,18,16,0.888889,0.888889
17302615,3,1,0,0.000000,0.000000
3243536,2,1,1,0.500000,1.000000
3248240,4,3,3,0.750000,1.000000


In [353]:
df_covering_overview['%_in_dict2bibs'].mean()

0.42435555470515196

In [354]:
df_covering_overview['%_in_dict2bibs_w_l'].mean()

0.5696041037576695

Мб добавить учет None и прогнать на реальном датасете


либо подскачать данные